# API ---->  FOURSQUARE
Hacer requests de:
    - Starbucks
    - Airports
    - Schools
    - Bar
    - Vegan

In [39]:
from ..src.contants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json
import numpy as np

ImportError: attempted relative import with no known parent package

In [2]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('dbcompanies','companies_clean')

In [4]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }}) 

In [5]:
data_companies = pd.DataFrame(companies)

In [6]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [7]:
def near(geo_point, radio_meters):
        geopoint = geo_point
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

radio_meters = 3000
number_offices = []
list_offices=[]
for i in range(len(data_companies)):
    num_offices =  near(geo_point[i], radio_meters)
    list_offices.append(num_offices)
    number_offices.append(len(num_offices))
data_companies['number offices near'] = number_offices

In [8]:
data_companies = data_companies[data_companies['number offices near'] > 1]

In [9]:
data_companies.head()

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number offices near
0,5e049e9de6eb90cad2d707ad,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
2,5e049e9de6eb90cad2d707af,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
3,5e049e9de6eb90cad2d707b0,Skydeck,9.0,2012,web,52cdef7c4bab8bd675298509,4000000.0,Palo Alto,USA,37.564538,-122.325470,"{'type': 'Point', 'coordinates': [-122.3254705...",2
4,5e049e9de6eb90cad2d707b1,PeekYou,20.0,2012,web,52cdef7c4bab8bd675297f94,1830000.0,New York,USA,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",6
9,5e049e9de6eb90cad2d707b7,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",2


In [10]:
data_companies.shape

(22, 13)

In [136]:
data_companies.to_json('./best_companies.json', orient='records',default_handler=str)

In [137]:
data_companies.to_csv('./best_companies.csv') 

In [11]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [12]:
 # Latitud 37
# Longitud -122


In [29]:
def getNearby(lat,lng,query,radius):
    """ Function for queries """
    CLIENT_ID = os.getenv("client_id")
    CLIENT_SECRET = os.getenv("client_secret")
    url = 'https://api.foursquare.com/v2/venues/explore'
    queryParams = dict(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        v='20181023', # DONDE ESTOY
        ll=f"{lat},{lng}", #QUE QUIERO
        query=query, # A QUE DISTANCIA
        radius=radius,
        limit = 25
    )
    res = requests.get(url, params=queryParams)
    return res.json()
  

In [30]:
#  Hacer las queries a todas las coordenadas
"""p = []
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
            p.append(request)
        except:
            Exception: f"Ha habido problemas con {place}"
"""

In [465]:
def curateResponse(data):
    """ Function for clean request and clean errors """
    places = []
    for item in data:
        try:
            for i in item["response"]["groups"][0]["items"]:
                if i["venue"]['location']['city'] != None:
                    places.append({
                        "name":i["venue"]['name'],
                        "distance": i["venue"]['location']["distance"],
                        "country": i["venue"]['location']["country"],
                        "city": i["venue"]['location']['city'],
                        "latitude": i["venue"]['location']["lat"],
                        "longitude": i["venue"]['location']["lng"],
                        "coordinates": {
                        "type":"Point",
                        "coordinates":[i["venue"]['location']["lng"], i["venue"]['location']["lat"]]
                            }})
        except:
            print(f"Ha ocurrido algún error.")
    return places

In [499]:
data = pd.DataFrame(curateResponse(p))

Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.


In [584]:
"""data.name.unique()"""

'data.name.unique()'

In [501]:
data.name.value_counts().head()

Starbucks        360
sweetgreen        13
Abokado           11
Juice Press       11
Us Helicopter      8
Name: name, dtype: int64

In [502]:
# Vamos a organizar los datos, y solo nos quedaremos en la columna 'type' con los valores 
# del diccionario <groups> 
groups= {
    "Coffee Shop": 'Starbucks',
    "Airport":['San Carlos Airport (SQL) (San Carlos Airport)',"Palo Alto Airport (PAO) (Palo Alto Airport)"
                "Santa Monica Airport (SMO) (Santa Monica Airport)","London City Airport (LCY) (London City Airport)",
                "St. Petersburg - Clearwater International Airport (PIE)",
                "Stockholm-Bromma Airport (BMA) (Stockholm-Bromma Airport)",
                "San Francisco International Airport (SFO) (San Francisco International Airport)",
                "LaGuardia Airport (LGA) (LaGuardia Airport)"],
    "Bar":["Bar","Nightcub","Cocktail Bar","Pub","Plunge Rooftop Bar & Lounge","Audio Nightclub","Temple Nightclub",
            "Momofuku Ssäm Bar","Absinthe Brasserie & Bar","Cue Bar","Ritz Bar & Lounge"],
    "School": ["Annex Stockholm International School","Petite Sorbonne Preschool","Trinity Preschool","Preschool of America","School","MS 297 Middle School",
               "Village Community School","Chelsea School","Public School 3","St. Matthew Catholic School",
              "De School Van Mieke Petiet","Goethe Instituut","Altra College","AltSchool",
               "Alta Vista School","Mission Montessori","Step by Step Early Childhood",
               "Grace Church School","Greenwich Village School","Santa Monica High School",
               "Stuyvesant High School","Norman Thomas High School","North Shoreview Montessori",
               "Highland Montessori","St. Timothy Catholic School","High Point Elementary School"],
    "Vegan": ["Thai Vegan ","Maoz Vegan" , "Urban Vegan Kitchen", "Golden Era Vegan", 
              "Vegan Junk Food Bar","Garden Fresh Vegan Cuisine ","Elovate Vegan Kitchen","Vegetarian Oasis","Maoz"]
}

def chooseCat(prod):
    for groupName, groupItems in groups.items():
        if prod in groupItems:
            return groupName
    return "OTHER"

In [503]:
data["category"] = data.name.apply(chooseCat)

In [504]:
data[data.category == 'Airport']

,name,distance,country,city,latitude,longitude,coordinates,category
25,San Francisco International Airport (SFO) (San...,7864,United States,San Francisco,37.616713,-122.387094,"{'type': 'Point', 'coordinates': [-122.3870944...",Airport
26,San Carlos Airport (SQL) (San Carlos Airport),8619,United States,San Carlos,37.515107,-122.251278,"{'type': 'Point', 'coordinates': [-122.2512780...",Airport
182,San Francisco International Airport (SFO) (San...,7954,United States,San Francisco,37.616713,-122.387094,"{'type': 'Point', 'coordinates': [-122.3870944...",Airport
183,San Carlos Airport (SQL) (San Carlos Airport),8553,United States,San Carlos,37.515107,-122.251278,"{'type': 'Point', 'coordinates': [-122.2512780...",Airport
270,LaGuardia Airport (LGA) (LaGuardia Airport),9978,United States,East Elmhurst,40.772888,-73.868809,"{'type': 'Point', 'coordinates': [-73.86880874...",Airport
379,St. Petersburg - Clearwater International Airp...,1300,United States,Clearwater,27.906642,-82.691361,"{'type': 'Point', 'coordinates': [-82.69136106...",Airport
391,Stockholm-Bromma Airport (BMA) (Stockholm-Brom...,5532,Sverige,Bromma,59.355532,17.945482,"{'type': 'Point', 'coordinates': [17.945482053...",Airport
440,Stockholm-Bromma Airport (BMA) (Stockholm-Brom...,5532,Sverige,Bromma,59.355532,17.945482,"{'type': 'Point', 'coordinates': [17.945482053...",Airport
509,St. Petersburg - Clearwater International Airp...,1300,United States,Clearwater,27.906642,-82.691361,"{'type': 'Point', 'coordinates': [-82.69136106...",Airport
793,London City Airport (LCY) (London City Airport),9971,United Kingdom,London,51.503980,0.049679,"{'type': 'Point', 'coordinates': [0.0496786870...",Airport


In [505]:
data.category.unique()

array(['Coffee Shop', 'Airport', 'OTHER', 'School', 'Bar', 'Vegan'],
      dtype=object)

In [563]:
def selector(row):
    if row['category'] == 'Coffee Shop':
        if row['distance'] <= 200:
            return 1
        elif row['distance'] > 200 and row['distance'] <= 500:
            return 2
        elif row['distance'] > 500 and row['distance'] <= 1000:
            return 3
        elif  row['distance'] > 1000 and row['distance'] <= 2000:
            return 4
        elif  row['distance'] > 2500 and row['distance'] <= 3000:
            return 5
        else:
            return 6
    if row['category'] == 'Airport':
        if row['distance'] <= 2000:
            return 1
        elif  row['distance'] > 2000 and row['distance'] <= 5000:
            return 2
        elif row['distance'] > 5000 and row['distance'] <= 10000:
            return 3
        elif  row['distance'] > 10000 and row['distance'] <= 20000:
            return 4
        elif row['distance'] > 20000:
            return 5
    if row['category'] == 'Vegan':
        if row['distance'] <= 300:
            return 1
        elif  row['distance'] > 300 and row['distance'] <= 500:
            return 2
        elif row['distance'] > 500 and row['distance'] <= 1000:
            return 3
        elif  row['distance'] > 1000 and row['distance'] <= 2000:
            return 4
        elif row['distance'] > 2000:
            return 5
    if row['category'] == 'School':
        if row['distance'] <= 500:
            return 1
        elif  row['distance'] > 500 and row['distance'] <= 1000:
            return 2
        elif row['distance'] > 1000 and row['distance'] <= 1500:
            return 3
        elif  row['distance'] > 2000 and row['distance'] <= 3000:
            return 4
        elif row['distance'] > 3000:
            return 5

In [564]:
data['score'] = data.apply(lambda row : selector(row), axis=1)

In [570]:
data.head(-10)

,name,distance,country,city,latitude,longitude,coordinates,category,score
0,Starbucks,444,United States,San Mateo,37.562890,-122.324005,"{'type': 'Point', 'coordinates': [-122.324005,...",Coffee Shop,2.0
1,Starbucks,1665,United States,San Mateo,37.579827,-122.333364,"{'type': 'Point', 'coordinates': [-122.3333638...",Coffee Shop,4.0
2,Starbucks,1853,United States,San Mateo,37.552260,-122.313842,"{'type': 'Point', 'coordinates': [-122.3138416...",Coffee Shop,4.0
3,Starbucks,2413,United States,Burlingame,37.579379,-122.346253,"{'type': 'Point', 'coordinates': [-122.3462526...",Coffee Shop,6.0
4,Starbucks,3677,United States,Foster City,37.558740,-122.283508,"{'type': 'Point', 'coordinates': [-122.2835075...",Coffee Shop,6.0
...,...,...,...,...,...,...,...,...,...
1776,Jack’s Stir Brew Coffee,23,United States,New York,40.740957,-74.006975,"{'type': 'Point', 'coordinates': [-74.00697509...",OTHER,NaN
1777,Blossom Restaurant,665,United States,New York,40.745394,-74.002106,"{'type': 'Point', 'coordinates': [-74.00210631...",OTHER,NaN
1778,juice press,419,United States,New York,40.739143,-74.002711,"{'type': 'Point', 'coordinates': [-74.00271122...",OTHER,NaN
1779,Liquiteria,438,United States,New York,40.740394,-74.001995,"{'type': 'Point', 'coordinates': [-74.00199476...",OTHER,NaN


In [571]:
data.drop(data.loc[data['score']> 4].index, inplace=True)

In [572]:
data.shape

(1661, 9)

In [573]:
data_new = data

In [574]:
data_new.score.value_counts()

3.0    164
2.0     79
1.0     26
4.0     15
Name: score, dtype: int64

In [575]:
data_new.category.value_counts()

OTHER          1367
Coffee Shop     230
School           35
Airport          10
Bar              10
Vegan             9
Name: category, dtype: int64

In [576]:
data_new.drop(data_new.loc[data_new.category == 'OTHER'].index, inplace=True)

In [577]:
data_new = data.reset_index(drop=True)

In [578]:
data_new.country.unique()

array(['United States', 'Sverige', 'United Kingdom', 'Nederland'],
      dtype=object)

In [579]:
data_new.loc[(data_new['country']== 'United States'), 'country'] = 'USA'
data_new.loc[(data_new['country']== 'Sverige'), 'country'] = 'SWE'
data_new.loc[(data_new['country']== 'United Kingdom'), 'country']  = 'GBR'
data_new.loc[(data_new['country']== 'Nederland'), 'country']  = 'NLD'


In [580]:
data_new.columns

Index(['name', 'distance', 'country', 'city', 'latitude', 'longitude',
       'coordinates', 'category', 'score'],
      dtype='object')

In [581]:
data_new = data_new[["name","country","city", "coordinates","longitude","latitude","category", "score"]]

In [582]:
data_new.category.unique()

array(['Coffee Shop', 'Airport', 'School', 'Bar', 'Vegan'], dtype=object)

In [583]:
data_new.to_json('./output/queries.json', orient='records',default_handler=str)